<a href="https://colab.research.google.com/github/PuchToTalk/ConsumerEdgeHackaton/blob/data/Data_extraction_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install wikipedia-api


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=a4a076887d99c0016caf19c91867fa0bb30481e33df4ad648b6b031ca3bc394e
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


## Simple Extraction from Wikipedia


In [2]:
import requests
import csv
from bs4 import BeautifulSoup

def get_wikipedia_data_csv(title, language="en"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Écriture dans le fichier CSV
            with open('wikipedia_data.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([page_title, page_url, article_extract, word_count, internal_links, categories, images])
    else:
        print("Erreur lors de la récupération des données.")

# Initialiser le fichier CSV avec les en-têtes
with open('wikipedia_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"])

# Appeler la fonction pour obtenir les détails de la page "Christmas" et les enregistrer dans un fichier CSV
get_wikipedia_data_csv("Christmas")
get_wikipedia_data_csv("Artificial intelligence")


# Initial Dataset from list of children's stories

In [11]:
import requests
import csv

def verify_wikipedia_pages(titles, language="fr"):
    # URL de base de l'API Wikipedia
    base_url = f"https://{language}.wikipedia.org/w/api.php"

    # Initialiser le fichier CSV avec les résultats
    with open('wikipedia_pages_verification.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["title", "exists", "wikipedia_url"])

        # Vérifier chaque titre
        for title in titles:
            params = {
                "action": "query",
                "format": "json",
                "titles": title
            }

            # Requête à l'API Wikipedia
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                data = response.json()
                page_id = next(iter(data['query']['pages']))

                if page_id == "-1":
                    # La page n'existe pas
                    writer.writerow([title, False, "N/A"])
                else:
                    # La page existe
                    page_url = f"https://{language}.wikipedia.org/wiki/{title.replace(' ', '_')}"
                    writer.writerow([title, True, page_url])
            else:
                print(f"Erreur lors de la vérification du titre '{title}'.")

# Liste des titres à vérifier
titles = [
    "Alice au Pays des Merveilles", "Peter Pan", "Robin des Bois", "Mulan", "Pocahontas",
    "Le Livre de la Jungle", "Bambi", "Le Roi Lion", "Les Aventures de Winnie l’Ourson",
    "La Princesse et la Grenouille", "La Reine des Neiges", "Moana (Vaiana)", "Blanche-Neige et Rose-Rouge",
    "Ali Baba et les Quarante Voleurs", "Sinbad le Marin", "Le Petit Poucet", "Peau d’Âne",
    "Les Aventures de Tintin", "Astérix et Obélix", "Le Magicien d’Oz", "Oliver Twist",
    "Les Douze Princesses Dansantes", "Le Joueur de Flûte de Hamelin", "La Princesse au Petit Pois",
    "Les Musiciens de Brême", "Ali et le Génie", "Anastasia", "La Petite Fille aux Allumettes",
    "Les Cygnes Sauvages", "Madame Bovary (version simplifiée)", "Heidi", "L’Étrange Noël de Monsieur Jack",
    "Wallace et Gromit", "Chicken Run", "Madagascar", "Kung Fu Panda", "Toy Story",
    "Le Monde de Nemo", "Les Indestructibles", "Shrek", "Cars", "Ratatouille", "Monstres et Cie",
    "Coco", "Les Aventures de Tom Sawyer", "Les Quatre Filles du Docteur March", "Mary Poppins",
    "La Mélodie du Bonheur", "Pippi Longstocking (Fifi Brindacier)", "Peter et Elliott le Dragon",
    "Ferdinand", "Le Petit Prince", "Taram et le Chaudron Magique", "Balto", "Spirit, l’Étalon des Plaines",
    "Les Aristochats", "Le Renard et l’Enfant", "Les Animaux du Bois de Quat’sous", "Bernard et Bianca",
    "Tinker Bell", "Frère des Ours", "Lilo et Stitch", "Rio", "Brave (Rebelle)", "Horton Hears a Who!",
    "Les Schtroumpfs", "Zootopia", "Les Trolls", "La Garde du Roi Lion", "La Légende de Korra",
    "Le Petit Nicolas", "Alice de l’Autre Côté du Miroir", "Charlie et la Chocolaterie", "Le Géant de Fer",
    "Mia et le Migou", "Les Enfants Loups, Ame et Yuki", "Ponyo sur la falaise", "Mon voisin Totoro",
    "Kiki la petite sorcière", "Le Voyage de Chihiro", "Princesse Mononoké", "Le Château ambulant",
    "Nausicaä de la vallée du vent", "Les Enfants de la mer", "Le Château dans le ciel", "Coraline",
    "Anastasia", "Ernest et Célestine", "Paddington", "Le Gruffalo", "Max et les Maximonstres",
    "Le Roi et l’Oiseau", "Kirikou et la Sorcière", "Azur et Asmar", "L’Histoire sans Fin",
    "La Fabuleuse Gilly Hopkins", "Un Chant de Noël", "Le Secret de la Petite Sirène", "Les Trois Brigands",
    "Le Renard et l’Érable"
]

# Vérifier chaque page Wikipedia
verify_wikipedia_pages(titles)


In [13]:
import os

# Get the current working directory
current_path = os.getcwd()
print(current_path)


/content


## Fixing missing values

In [18]:
import csv

# URLs fournis à ajouter
additional_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)" : "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde"
    "Coco" : "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification.csv'
output_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'

# Mise à jour du fichier CSV avec les URLs supplémentaires
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs supplémentaires si nécessaires
    for row in reader:
        title = row['title']
        if title in additional_urls and (row['wikipedia_url'] == "N/A" or not row['wikipedia_url']):
            row['exists'] = True
            row['wikipedia_url'] = additional_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path


'/content/wikipedia_pages_verification_updated_final.csv'

## Generate the corpus & Merge results

In [21]:
import requests
import csv
from bs4 import BeautifulSoup

# Fonction pour obtenir les détails de Wikipédia
def get_wikipedia_data_csv(title, language="fr"):
    # URL de l'API Wikipedia
    url = f"https://{language}.wikipedia.org/w/api.php"

    # Paramètres pour extraire les différentes informations de la page
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|links|categories|images|info",
        "exintro": True,
        "titles": title,
        "pllimit": "max",
        "cllimit": "max",
        "inprop": "url"
    }

    # Requête à l'API Wikipedia
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        page_id = next(iter(data['query']['pages']))
        if page_id == "-1":
            print(f"La page '{title}' n'existe pas.")
        else:
            page = data['query']['pages'][page_id]

            # Extraction des données de l'article
            page_title = page.get('title', 'N/A')
            page_url = page.get('fullurl', 'N/A')

            # Extraire et nettoyer le texte de l'article
            raw_extract = page.get('extract', 'N/A')
            if raw_extract != 'N/A':
                soup = BeautifulSoup(raw_extract, "html.parser")
                article_extract = soup.get_text(strip=True)  # Supprime toutes les balises HTML
            else:
                article_extract = 'N/A'

            # Calcul du nombre de mots dans l'extrait nettoyé
            word_count = len(article_extract.split()) if article_extract != 'N/A' else 0

            # Extraction des liens internes
            internal_links = "; ".join([link['title'] for link in page.get('links', []) if 'title' in link])

            # Extraction des catégories
            categories = "; ".join([category['title'] for category in page.get('categories', []) if 'title' in category])

            # Extraction des images (nom de fichier seulement)
            images = "; ".join([image['title'] for image in page.get('images', []) if 'title' in image])

            # Retourner les données sous forme de dictionnaire
            return {
                "page_title": page_title,
                "page_url": page_url,
                "article_extract": article_extract,
                "word_count": word_count,
                "internal_links": internal_links,
                "categories": categories,
                "images": images
            }
    else:
        print("Erreur lors de la récupération des données.")
        return None

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/wikipedia_pages_verification_updated_final.csv'
output_csv_path = '/content/wikipedia_data_enriched.csv'

# Lire le fichier CSV et appliquer la fonction à chaque titre
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        title = row['title']
        if row['exists'].lower() == 'true':
            wikipedia_data = get_wikipedia_data_csv(title)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Chemin du fichier CSV enrichi
output_csv_path


La page 'Les Aventures de Winnie l’Ourson' n'existe pas.
La page 'La Reine des Neiges' n'existe pas.
La page 'Moana (Vaiana)' n'existe pas.
La page 'Les Douze Princesses Dansantes' n'existe pas.
La page 'Le Joueur de Flûte de Hamelin' n'existe pas.
La page 'La Princesse au Petit Pois' n'existe pas.
La page 'Ali et le Génie' n'existe pas.
La page 'La Petite Fille aux Allumettes' n'existe pas.
La page 'Les Cygnes Sauvages' n'existe pas.
La page 'Madame Bovary (version simplifiée)' n'existe pas.
La page 'L’Étrange Noël de Monsieur Jack' n'existe pas.
La page 'La Mélodie du Bonheur' n'existe pas.
La page 'Pippi Longstocking (Fifi Brindacier)' n'existe pas.
La page 'Taram et le Chaudron Magique' n'existe pas.
La page 'Spirit, l’Étalon des Plaines' n'existe pas.
La page 'Le Renard et l’Enfant' n'existe pas.
La page 'Brave (Rebelle)' n'existe pas.
La page 'Alice de l’Autre Côté du Miroir' n'existe pas.
La page 'Le Géant de Fer' n'existe pas.
La page 'Les Enfants Loups, Ame et Yuki' n'existe p

'/content/wikipedia_data_enriched.csv'

In [31]:
import csv

# URLs fournis à ajouter
replacement_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Les_Aventures_d%27Alice_au_pays_des_merveilles",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Le_Roi_lion_(film,_1994)",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Peau_d%27Âne",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Astérix",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/Le_Magicien_d%27Oz",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Peter_et_Elliott_le_dragon",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/Les_Animaux_du_Bois_de_Quat%27sous",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/Les_Aventures_de_Bernard_et_Bianca",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fée_Clochette",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Frère_des_ours",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Horton_(film)",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Zootopie",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/La_Garde_du_Roi_lion",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Un_chant_de_Noël",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille":"https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)"
}

# Chemin du fichier CSV téléchargé
input_csv_path = '/content/final-wiki.csv'
output_csv_path = '/content/wiki1.csv'

# Mise à jour du fichier CSV avec les URLs de remplacement
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les URLs de remplacement si nécessaire
    for row in reader:
        title = row['title']
        if title in replacement_urls:
            row['exists'] = True
            row['wikipedia_url'] = replacement_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

'/content/wiki1.csv'

In [ ]:
import csv
import sys

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# URLs fournies à ajouter ou mettre à jour
new_urls = {
    "Les Aventures de Winnie l’Ourson": "https://fr.wikipedia.org/wiki/Winnie_l%27ourson",
    "La Reine des Neiges": "https://fr.wikipedia.org/wiki/La_Reine_des_neiges",
    "Les Douze Princesses Dansantes": "https://fr.wikipedia.org/wiki/Le_Bal_des_douze_princesses",
    "Le Joueur de Flûte de Hamelin": "https://fr.wikipedia.org/wiki/Le_Joueur_de_fl%C3%BBte_de_Hamelin",
    "La Princesse au Petit Pois": "https://fr.wikipedia.org/wiki/La_Princesse_au_petit_pois",
    "Ali et le Génie": "https://fr.wikipedia.org/wiki/Aladdin_(Disney)",
    "La Petite Fille aux Allumettes": "https://fr.wikipedia.org/wiki/La_Petite_Fille_aux_allumettes",
    "Les Cygnes Sauvages": "https://fr.wikipedia.org/wiki/Les_Cygnes_sauvages_(Andersen)",
    "Madame Bovary (version simplifiée)": "https://fr.wikipedia.org/wiki/Madame_Bovary",
    "L’Étrange Noël de Monsieur Jack": "https://fr.wikipedia.org/wiki/L%27%C3%89trange_No%C3%ABl_de_monsieur_Jack",
    "La Mélodie du Bonheur": "https://fr.wikipedia.org/wiki/La_M%C3%A9lodie_du_bonheur_(film,_1965)",
    "Pippi Longstocking (Fifi Brindacier)": "https://fr.wikipedia.org/wiki/Fifi_Brindacier",
    "Taram et le Chaudron Magique": "https://fr.wikipedia.org/wiki/Taram_et_le_Chaudron_magique",
    "Spirit, l’Étalon des Plaines": "https://fr.wikipedia.org/wiki/Spirit,_l%27%C3%A9talon_des_plaines",
    "Le Renard et l’Enfant": "https://fr.wikipedia.org/wiki/Le_Renard_et_l%27Enfant",
    "Brave (Rebelle)": "https://fr.wikipedia.org/wiki/Rebelle_(film_d%27animation,_2012)",
    "Alice de l’Autre Côté du Miroir": "https://fr.wikipedia.org/wiki/Alice_de_l%27autre_c%C3%B4t%C3%A9_du_miroir",
    "Le Géant de Fer": "https://fr.wikipedia.org/wiki/Le_G%C3%A9ant_de_fer",
    "Les Enfants Loups, Ame et Yuki": "https://fr.wikipedia.org/wiki/Les_Enfants_loups,_Ame_et_Yuki",
    "Le Roi et l’Oiseau": "https://fr.wikipedia.org/wiki/Le_Roi_et_l%27Oiseau",
    "L’Histoire sans Fin": "https://fr.wikipedia.org/wiki/L%27Histoire_sans_fin_(film)",
    "Le Renard et l’Érable": "https://fr.wikipedia.org/wiki/Le_Renard_et_les_Raisins",
    "Moana (Vaiana)": "https://fr.wikipedia.org/wiki/Vaiana_:_La_L%C3%A9gende_du_bout_du_monde",
    "Coco": "https://fr.wikipedia.org/wiki/Coco_(film,_2017)",
    "Les Trolls": "https://fr.wikipedia.org/wiki/Les_Trolls_(film)",
    "Les Enfants de la mer": "https://fr.wikipedia.org/wiki/Les_Enfants_de_la_mer_(film)",
    "Nausicaä de la Vallée du Vent": "https://fr.wikipedia.org/wiki/Nausicaä_de_la_Vallée_du_Vent_(film_d%27animation)",
    "Ratatouille": "https://fr.wikipedia.org/wiki/Ratatouille_(film)",
    "Cars": "https://fr.wikipedia.org/wiki/Cars_(film)",
    "Shrek": "https://fr.wikipedia.org/wiki/Shrek_(film)"
}

# URLs supplémentaires à mettre à jour
additional_urls = {
    "Bambi": "https://fr.wikipedia.org/wiki/Bambi_(film,_1942)",
    "Le Livre de la jungle": "https://fr.wikipedia.org/wiki/Le_Livre_de_la_jungle_(film,_1967)",
    "Anastasia": "https://fr.wikipedia.org/wiki/Anastasia_(film,_1997)",
    "Heidi": "https://fr.wikipedia.org/wiki/Heidi_(série_télévisée_d%27animation)",
    "Mary Poppins": "https://fr.wikipedia.org/wiki/Mary_Poppins_(film)",
    "Les Quatre Filles du docteur March (homonymie)": "https://fr.wikipedia.org/wiki/Les_Quatre_Filles_du_docteur_March",
    "Les Trois Brigands": "https://fr.wikipedia.org/wiki/Les_Trois_Brigands_(film)"
}

# Fusionner toutes les nouvelles URLs
all_urls = {**new_urls, **additional_urls}

# Chemin du fichier CSV téléchargé
input_csv_path = '/mnt/data/first-result.csv'
output_csv_path = '/mnt/data/first-result-overwritten-final-complete.csv'

# Mise à jour du fichier CSV avec les nouvelles URLs en utilisant la colonne 'page_title' pour remplacer la colonne 'page_url'
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Mise à jour des lignes existantes avec les nouvelles URLs pour 'page_url'
    for row in reader:
        title = row['page_title']
        if title in all_urls:
            row['page_url'] = all_urls[title]
        writer.writerow(row)

# Chemin du fichier CSV mis à jour
output_csv_path

In [35]:
from google.colab import files
import requests
import csv
import sys
from bs4 import BeautifulSoup

# Étape 1: Charger le fichier CSV
uploaded = files.upload()  # Téléchargez votre fichier CSV ici.

# Étape 2: Mettre à jour le chemin du fichier CSV avec le nom du fichier téléchargé
input_csv_path = '/content/step1-update.csv'  # Remplacez par le nom de votre fichier téléchargé
output_csv_path = '/content/final-step1.csv'

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# Étape 3: Fonction pour extraire les détails de Wikipédia
def get_wikipedia_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Utiliser BeautifulSoup pour extraire le contenu de la page Wikipedia
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le titre de la page
        page_title = soup.find("h1", {"id": "firstHeading"}).text if soup.find("h1", {"id": "firstHeading"}) else "N/A"

        # Extraire le texte de l'article (principalement à partir de balises <p>)
        paragraphs = soup.find_all("p")
        article_extract = " ".join([p.text for p in paragraphs])
        article_extract = article_extract.strip()

        # Calcul du nombre de mots dans l'extrait
        word_count = len(article_extract.split()) if article_extract else 0

        # Extraire les liens internes (balises <a> dans le corps de l'article)
        internal_links = []
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                internal_links.append(link.text.strip())
        internal_links_str = "; ".join(internal_links)

        # Extraire les catégories (depuis la section des catégories)
        categories = []
        for category in soup.find_all("div", {"id": "mw-normal-catlinks"}):
            for link in category.find_all("a"):
                categories.append(link.text.strip())
        categories_str = "; ".join(categories)

        # Extraire les images
        images = []
        for img in soup.find_all("img"):
            images.append(img.get("src"))
        images_str = "; ".join(images)

        # Retourner les données sous forme de dictionnaire
        return {
            "page_title": page_title,
            "page_url": url,
            "article_extract": article_extract,
            "word_count": word_count,
            "internal_links": internal_links_str,
            "categories": categories_str,
            "images": images_str
        }
    else:
        print(f"Erreur lors de la récupération de la page : {url}")
        return None

# Étape 4: Lire le fichier CSV et appliquer la fonction à chaque URL
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["page_title", "page_url", "article_extract", "word_count", "internal_links", "categories", "images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        url = row['page_url']
        if url and url != "N/A":
            wikipedia_data = get_wikipedia_data_from_url(url)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Étape 5: Télécharger le fichier CSV enrichi
files.download(output_csv_path)

Saving step1-update.csv to step1-update (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now that we have the Data "final-step1.csv", we use it to generate the concise text in another column using OpenAI API

In [108]:
from openai import OpenAI

# Pass the API key directly to the OpenAI constructor
# Load the API key from the text file
with open('/content/api_key.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client using the loaded API key
client = OpenAI(api_key=api_key)



completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [50]:





completion = client.chat.completions.create(
    model = "gpt-4o",
    messages = [
        {"role": "system", "content": "You are a helpful assistant who generates child-friendly, concise summaries of stories."},
        {"role": "user", "content": (
            f"Générer dans une autre colonne l'histoire du conte / de l'oeuvre '{title}' "
            f"et le raconter d'une façon à ce qu'on s'adresse à l'enfant de manière pas trop longue. "
            f"Bien narrer de manière adaptée pour un enfant de bas âge. Une explication concise.\n\n"
            f"Voici l'extrait de la page Wikipédia :\n\n{text}"
        )}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Il était une fois un petit prince qui vivait sur une toute petite planète avec une rose. Un jour, il décida de partir explorer l'univers et rencontra plein de personnages intéressants, comme un roi, un homme d'affaires, et un allumeur de réverbères. Chaque rencontre lui apprend quelque chose de nouveau.\n\nEnsuite, il arrive sur Terre, où il devient ami avec un aviateur perdu dans le désert. Le petit prince découvre aussi un renard, qui lui révèle l'importance de l'amitié et du fait d'apprivoiser quelqu'un. Grâce à ses aventures, le petit prince comprend que l'amour et les liens sont ce qui compte le plus dans la vie. \n\nÀ la fin, le petit prince décide de retourner sur sa planète pour être avec sa rose, mais ses enseignements sur l'amitié et l'amour restent gravés dans le cœur de l'aviateur et de tous ceux qui entendent son histoire.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


## ENGLISH DATASET

In [70]:
import pandas as pd

# List of titles
titles_list = [
    "Alice's Adventures in Wonderland",
    "Peter Pan",
    "Robin Hood",
    "Mulan",
    "Pocahontas",
    "The Jungle Book",
    "Bambi",
    "The Lion King",
    "Winnie-the-Pooh",
    "The Princess and the Frog",
    "Frozen",
    "Moana",
    "Snow-White and Rose-Red",
    "Ali Baba and the Forty Thieves",
    "Sinbad the Sailor",
    "Hop-o'-My-Thumb",
    "Donkeyskin",
    "The Adventures of Tintin",
    "Asterix and Obelix",
    "The Wizard of Oz",
    "Oliver Twist",
    "The Twelve Dancing Princesses",
    "The Pied Piper of Hamelin",
    "The Princess and the Pea",
    "The Bremen Town Musicians",
    "Aladdin",
    "Anastasia",
    "The Little Match Girl",
    "The Wild Swans",
    "Madame Bovary",
    "Heidi",
    "The Nightmare Before Christmas",
    "Wallace and Gromit",
    "Chicken Run",
    "Madagascar",
    "Kung Fu Panda",
    "Toy Story",
    "Finding Nemo",
    "The Incredibles",
    "Shrek",
    "Cars",
    "Ratatouille",
    "Monsters, Inc.",
    "Coco",
    "The Adventures of Tom Sawyer",
    "Little Women",
    "Mary Poppins",
    "The Sound of Music",
    "Pippi Longstocking",
    "Pete's Dragon",
    "Ferdinand",
    "The Little Prince",
    "The Black Cauldron",
    "Balto",
    "Spirit: Stallion of the Cimarron",
    "The Aristocats",
    "The Fox and the Child",
    "Animals of Farthing Wood",
    "The Rescuers",
    "Tinker Bell",
    "Brother Bear",
    "Lilo & Stitch",
    "Rio",
    "Brave",
    "Horton Hears a Who!",
    "The Smurfs",
    "Zootopia",
    "Trolls",
    "The Lion Guard",
    "The Legend of Korra",
    "Le Petit Nicolas",
    "Alice Through the Looking Glass",
    "Charlie and the Chocolate Factory",
    "The Iron Giant",
    "Mia and the Migoo",
    "Wolf Children",
    "Ponyo",
    "My Neighbor Totoro",
    "Kiki's Delivery Service",
    "Spirited Away",
    "Princess Mononoke",
    "Howl's Moving Castle",
    "Nausicaä of the Valley of the Wind",
    "Children of the Sea",
    "Castle in the Sky",
    "Coraline",
    "Anastasia",
    "Ernest & Celestine",
    "Paddington",
    "The Gruffalo",
    "Where the Wild Things Are",
    "The King and the Mockingbird",
    "Kirikou and the Sorceress",
    "Azur & Asmar",
    "The NeverEnding Story",
    "The Great Gilly Hopkins",
    "A Christmas Carol",
    "The Little Mermaid",
    "The Three Robbers",
    "The Fox and the Maple"
]

# Function to create a formatted Wikipedia link part from the translated title
def create_formatted_title(title):
    # Replace each single quote with %27 and spaces with underscores
    formatted_title = title.replace("'", "%27").replace(" ", "_")
    return formatted_title

# Generate formatted titles
formatted_titles = [create_formatted_title(title) for title in titles_list]

# Create the full URLs from the formatted titles
english_urls = [f"https://en.wikipedia.org/wiki/{formatted_title}" for formatted_title in formatted_titles]

# Assume df is already loaded, add the english_urls as a new column
input_csv_path = '/content/final-step1.csv'
df = pd.read_csv(input_csv_path)

# Add the new column with the English Wikipedia URLs
df['english_url'] = english_urls

# Save the updated dataframe to a new CSV file
output_csv_path = '/content/final-step1-with-english-url.csv'
df.to_csv(output_csv_path, index=False)

print(f"CSV file updated and saved successfully: {output_csv_path}")

CSV file updated and saved successfully: /content/final-step1-with-english-url.csv


In [71]:
from google.colab import files
import requests
import csv
import sys
from bs4 import BeautifulSoup

# Étape 1: Charger le fichier CSV
uploaded = files.upload()  # Téléchargez votre fichier CSV ici.

# Étape 2: Mettre à jour le chemin du fichier CSV avec le nom du fichier téléchargé
input_csv_path = '/content/final-step1-with-english-url.csv'  # Remplacez par le nom de votre fichier téléchargé
output_csv_path = '/content/final-step1-with-english-text.csv'

# Augmenter la limite maximale de taille de champ
csv.field_size_limit(sys.maxsize)

# Étape 3: Fonction pour extraire les détails de Wikipédia
def get_wikipedia_data_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        # Utiliser BeautifulSoup pour extraire le contenu de la page Wikipedia
        soup = BeautifulSoup(response.text, "html.parser")

        # Extraire le titre de la page
        page_title = soup.find("h1", {"id": "firstHeading"}).text if soup.find("h1", {"id": "firstHeading"}) else "N/A"

        # Extraire le texte de l'article (principalement à partir de balises <p>)
        paragraphs = soup.find_all("p")
        article_extract = " ".join([p.text for p in paragraphs])
        article_extract = article_extract.strip()

        # Calcul du nombre de mots dans l'extrait
        word_count = len(article_extract.split()) if article_extract else 0

        # Extraire les liens internes (balises <a> dans le corps de l'article)
        internal_links = []
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            if href.startswith("/wiki/") and not href.startswith("/wiki/Special:"):
                internal_links.append(link.text.strip())
        internal_links_str = "; ".join(internal_links)

        # Extraire les catégories (depuis la section des catégories)
        categories = []
        for category in soup.find_all("div", {"id": "mw-normal-catlinks"}):
            for link in category.find_all("a"):
                categories.append(link.text.strip())
        categories_str = "; ".join(categories)

        # Extraire les images
        images = []
        for img in soup.find_all("img"):
            images.append(img.get("src"))
        images_str = "; ".join(images)

        # Retourner les données sous forme de dictionnaire
        return {
            "en_page_title": page_title,
            "en_page_url": url,
            "en_article_extract": article_extract,
            "en_word_count": word_count,
            "en_internal_links": internal_links_str,
            "en_categories": categories_str,
            "en_images": images_str
        }
    else:
        print(f"Erreur lors de la récupération de la page : {url}")
        return None

# Étape 4: Lire le fichier CSV et appliquer la fonction à chaque URL
with open(input_csv_path, mode='r', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ["en_page_title", "en_page_url", "en_article_extract", "en_word_count", "en_internal_links", "en_categories", "en_images"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier de sortie
    writer.writeheader()

    # Appliquer la fonction à chaque ligne
    for row in reader:
        url = row['english_url']
        if url and url != "N/A":
            wikipedia_data = get_wikipedia_data_from_url(url)
            if wikipedia_data:
                writer.writerow(wikipedia_data)

# Étape 5: Télécharger le fichier CSV enrichi
files.download(output_csv_path)

Saving final-step1-with-english-url.csv to final-step1-with-english-url (1).csv
Erreur lors de la récupération de la page : https://en.wikipedia.org/wiki/The_Fox_and_the_Maple


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:

# Define the function to rename the first columns
def rename_columns(input_csv_path, output_csv_path):
    # Read the CSV file using pandas
    df = pd.read_csv(input_csv_path)

    # Define the new fieldnames for the first columns
    new_fieldnames = ["fr_page_title", "fr_page_url", "fr_article_extract", "fr_word_count", "fr_internal_links", "fr_categories", "fr_images"]

    # Rename the first columns with the new fieldnames
    for i, new_name in enumerate(new_fieldnames):
        df.rename(columns={df.columns[i]: new_name}, inplace=True)

    # Save the updated dataframe to a new CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"CSV file with renamed columns saved successfully: {output_csv_path}")

# File paths
input_csv_path = '/content/fr_data.csv'
output_csv_path = '/content/new_fr_data.csv'

# Call the function to rename the columns and save the updated CSV
rename_columns(input_csv_path, output_csv_path)

CSV file with renamed columns saved successfully: /content/new_fr_data.csv


### Merge the Fr_dataset to En_dataset

In [74]:
import pandas as pd

# File paths for the input CSV files
fr_data_path = '/content/new_fr_data.csv'
en_data_path = '/content/en_data.csv'

# Read the French and English CSV files into dataframes
df_fr = pd.read_csv(fr_data_path)
df_en = pd.read_csv(en_data_path)

# Merge the dataframes on the specified columns
merged_df = pd.merge(df_fr, df_en, how='inner', left_on='english_url', right_on='en_page_url')

# Save the merged dataframe to a new CSV file
output_csv_path = '/content/merged_fr_en_data.csv'
merged_df.to_csv(output_csv_path, index=False)

print(f"Merged CSV file saved successfully: {output_csv_path}")

Merged CSV file saved successfully: /content/merged_fr_en_data.csv


## FUNCTION FOR GENERATING RELEVANT FORMAT

In [75]:
import pandas as pd

# Read the CSV file using pandas
input_csv_path = '/content/merged_fr_en_data.csv'
df = pd.read_csv(input_csv_path)
num_rows = len(df)  # Set the number of rows to process as the total number of rows

# Define a function to read the dataset and extract the columns as lists
def read_dataset(df, num_rows=5):
    # Extract the "article_extract" and "page_title" for the first `num_rows` rows
    text_list = df["en_article_extract"].tolist()[:num_rows]
    title_list = df["en_page_title"].tolist()[:num_rows]

    return title_list, text_list

# Read the dataset and get lists for the first `num_rows` rows
titles, texts = read_dataset(df, num_rows=num_rows)

# Print the extracted lists to verify
print(f"Titles: {titles}")
print(f"Texts: {texts}")

Titles: ["Alice's Adventures in Wonderland", 'Peter Pan', 'Robin Hood', 'Hua Mulan', 'Pocahontas', 'The Jungle Book', 'Bambi', 'The Lion King', 'Winnie-the-Pooh', 'The Princess and the Frog', 'Frozen', 'Moana', 'Snow-White and Rose-Red', 'Ali Baba and the Forty Thieves', 'Sinbad the Sailor', "Hop-o'-My-Thumb", 'Donkeyskin', 'The Adventures of Tintin', 'Asterix', 'The Wizard of Oz', 'Oliver Twist', 'The Twelve Dancing Princesses', 'Pied Piper of Hamelin', 'The Princess and the Pea', 'Town Musicians of Bremen', 'Aladdin', 'Anastasia', 'Anastasia', 'The Little Match Girl', 'The Wild Swans', 'Madame Bovary', 'Heidi', 'The Nightmare Before Christmas', 'Wallace & Gromit', 'Chicken Run', 'Madagascar', 'Kung Fu Panda', 'Toy Story', 'Finding Nemo', 'The Incredibles', 'Shrek', 'Car', 'Ratatouille', 'Monsters, Inc.', 'Coco', 'The Adventures of Tom Sawyer', 'Little Women', 'Mary Poppins', 'The Sound of Music', 'Pippi Longstocking', "Pete's Dragon", 'Ferdinand', 'The Little Prince', 'The Black Caul

In [78]:
#pip install tiktoken

In [80]:
import tiktoken
import time


# List to store the generated summaries
summaries = []
token_counts = []


# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")


# Loop over each title and text and generate summaries using OpenAI API
for i in range(num_rows):
    title = titles[i]
    extract = texts[i]

    # Create the messages format expected by ChatCompletion
    messages = [
        {"role": "system", "content": "You are a helpful assistant who generates child-friendly, concise summaries of stories."},
        {"role": "user", "content": (
            f"Générer dans une autre colonne l'histoire du conte / de l'oeuvre '{title}' "
            f"et le raconter d'une façon à ce qu'on s'adresse à l'enfant de manière pas trop longue. "
            f"Bien narrer de manière adaptée pour un enfant de bas âge. Une explication concise.\n\n"
            f"Voici l'extrait de la page Wikipédia :\n\n{extract}"
        )}
    ]

    # Token count for the messages
    total_tokens = 0
    for message in messages:
        message_tokens = len(enc.encode(message["content"]))
        total_tokens += message_tokens

    # Print the number of tokens used for the messages
    print(f"Total tokens for row {i + 1}: {total_tokens}")


    completion = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages
    )


    print(completion.choices[0].message)
    # Extract the generated summary
    summary = completion.choices[0].message.content.strip()
    summaries.append(summary)

    token_counts.append(total_tokens)

    # Add a delay after processing every 10 rows
    #if (i + 1) % 5 == 0:
    #    print("Processed 5 rows, waiting for 1 minute to avoid rate limits...")
    #    time.sleep(80)



Total tokens for row 1: 6226
ChatCompletionMessage(content='Voici une version simplifiée de l\'histoire d\'"Alice au pays des merveilles" pour les jeunes enfants :\n\nUn jour, une petite fille nommée Alice se promenait au bord d\'une rivière lorsqu\'elle aperçut un Lapin Blanc qui semblait très pressé. Curieuse, elle le suivit et tomba dans un terrier de lapin qui l\'amena dans un monde magique et étrange appelé le pays des merveilles. Là-bas, tout était à l\'envers, et les animaux pouvaient parler comme des gens !\n\nAlice fit la rencontre de nombreux personnages fantastiques : un gros chat souriant appelé le Chat du Cheshire, un Chapelier Fou qui aime le thé, et une Reine de Cœurs qui adore donner des ordres de couper la tête à tout le monde ! \n\nAlice grandit et rétrécit en mangeant et buvant des choses étranges. Elle participa même à une partie de croquet bizarre où les maillets étaient des flamants roses et les balles des hérissons !\n\nFinalement, Alice se retrouva face à la Rei

In [81]:
output_csv_path = '/content/final-step2-openai.csv'

# Add the summaries as a new column to the dataframe
df['concise_story'] = pd.Series(summaries, index=df.index[:num_rows])

# Save the updated dataframe to a new CSV file
df.to_csv(output_csv_path, index=False)

# Output confirmation
print(f"CSV file enriched successfully and saved to: {output_csv_path}")

CSV file enriched successfully and saved to: /content/final-step2-openai.csv


### Load & Filter the Extracted data

In [102]:
import pandas as pd
import re


output_openai = '/content/final-step2-openai.csv'

dff = pd.read_csv(output_openai)



# Define a function to remove characters like [\d+]
def remove_bracketed_numbers(text):
    return re.sub(r'\[\d+\]', '', text)

# Apply the function to the "en_article_extract" column
dff["en_article_extract"] = dff["en_article_extract"].apply(remove_bracketed_numbers)
dff["fr_article_extract"] = dff["fr_article_extract"].apply(remove_bracketed_numbers)

dff

,fr_page_title,fr_page_url,fr_article_extract,fr_word_count,fr_internal_links,fr_categories,fr_images,english_url,en_page_title,en_page_url,en_article_extract,en_word_count,en_internal_links,en_categories,en_images,concise_story
0,Les Aventures d'Alice au pays des merveilles,https://fr.wikipedia.org/wiki/Alice_au_Pays_de...,« Alice au pays des merveilles » redirige ici....,2706,Accueil; Portails thématiques; Article au hasa...,Catégories; Alice au pays des merveilles; Roma...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/Alice%27s_Advent...,Alice's Adventures in Wonderland,https://en.wikipedia.org/wiki/Alice%27s_Advent...,Alice's Adventures in Wonderland (also known a...,4455,Main page; Contents; Current events; About Wik...,Categories; 1865 British novels; 1865 fantasy ...,/static/images/icons/wikipedia.png; /static/im...,"\n\nUn jour, une petite fille nommée Alice se ..."
1,Peter Pan,https://fr.wikipedia.org/wiki/Peter_Pan,"Pour les articles homonymes, voir Peter Pan (h...",2996,Accueil; Portails thématiques; Article au hasa...,Catégories; Peter Pan; Œuvre de littérature d'...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/Peter_Pan,Peter Pan,https://en.wikipedia.org/wiki/Peter_Pan,Peter Pan is a fictional character created by ...,2927,Main page; Contents; Current events; About Wik...,Categories; Peter Pan; Peter Pan characters; F...,/static/images/icons/wikipedia.png; /static/im...,Peter Pan est un garçon courageux et curieux q...
2,Robin des Bois,https://fr.wikipedia.org/wiki/Robin_des_Bois,"Pour les articles homonymes, voir Robin des Bo...",1830,Accueil; Portails thématiques; Article au hasa...,Catégories; Robin des Bois; Archer de fiction;...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/Robin_Hood,Robin Hood,https://en.wikipedia.org/wiki/Robin_Hood,Robin Hood is a legendary heroic outlaw origin...,9907,Main page; Contents; Current events; About Wik...,Categories; Robin Hood; Robin Hood characters;...,/static/images/icons/wikipedia.png; /static/im...,Il était une fois un héros nommé Robin des Boi...
3,Mulan,https://fr.wikipedia.org/wiki/Mulan,Série Classiques d'animation Disney Hercule(19...,2238,Accueil; Portails thématiques; Article au hasa...,Catégories; Film américain sorti en 1998; Film...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/Mulan,Hua Mulan,https://en.wikipedia.org/wiki/Mulan,Hua Mulan (Chinese: 花木蘭) is a legendary Chines...,1974,Main page; Contents; Current events; About Wik...,Categories; Mulan; Chinese poems; Chinese warr...,/static/images/icons/wikipedia.png; /static/im...,"Il était une fois, dans la Chine ancienne, une..."
4,Pocahontas,https://fr.wikipedia.org/wiki/Pocahontas,"Pour les articles homonymes, voir Pocahontas (...",2284,Accueil; Portails thématiques; Article au hasa...,Catégories; Dirigeant politique du XVIIe siècl...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/Pocahontas,Pocahontas,https://en.wikipedia.org/wiki/Pocahontas,"Pocahontas (US: /ˌpoʊkəˈhɒntəs/ ⓘ, UK: /ˌpɒk-/...",3506,Main page; Contents; Current events; About Wik...,Categories; 1590s births; 1617 deaths; 17th-ce...,/static/images/icons/wikipedia.png; /static/im...,Il était une fois une jeune fille amérindienne...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,L'Histoire sans fin (film),https://fr.wikipedia.org/wiki/L%27Histoire_san...,"Pour les articles homonymes, voir L'Histoire s...",1355,Accueil; Portails thématiques; Article au hasa...,Catégories; L'Histoire sans fin; Film réalisé ...,/static/images/icons/wikipedia.png; /static/im...,https://en.wikipedia.org/wiki/The_NeverEnding_...,The Neverending Story,https://en.wikipedia.org/wiki/The_NeverEnding_...,The Neverending Story (German: Die unendliche ...,2306,Main page; Contents; Current events; About Wik...,Categories; The Neverending Story; 1979 childr...,/static/images/icons/wikipe

In [103]:
titles, texts = read_dataset(dff, num_rows=num_rows)

### Generating the narrative story

In [106]:
import tiktoken
import random
import time


# List to store the generated summaries
en_summaries = []
token_counts = []


# Initialize tiktoken encoder for the "gpt-4o" model
enc = tiktoken.encoding_for_model("gpt-4o")


# Loop over each title and text and generate summaries using OpenAI API
for i in range(num_rows):
    title = titles[i]
    extract = texts[i]

    # Create the messages format expected by ChatCompletion
    """
    messages = [
    {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories."},
    {"role": "user", "content": (
        f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
        f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children.\n\n"
        f"Here is an extract from the Wikipedia page:\n\n{extract}"
    )}
    ]
    """
    # Define a list of possible opening phrases to vary the beginning of the story
    opening_phrases = [
      "Long ago in a distant land,",
      "There was a time when",
      "In a magical kingdom,",
      "In a faraway place,",
      "Not too long ago,",
      "Deep in the enchanted forest,",
      "High on a mountain top,",
      "In a tiny village,",
      "In a beautiful land of wonder,",
      "In the middle of a bustling town,",
      "Once upon a time,"
    ]

    # Select a random opening phrase for variety
    opening = random.choice(opening_phrases)
    #print(opening)

    messages = [
        {"role": "system", "content": "You are a helpful assistant who creates child-friendly, engaging versions of stories. Do not start the story with 'Once upon a time'. Use creative and different beginnings instead."},
        {"role": "user", "content": (
            f"Reformulate the story of '{title}' into a children's tale, telling it in a way that is engaging and easy for young children to understand. "
            f"Make sure the story feels like an enchanting bedtime tale, with a simple and friendly tone suitable for young children. Start with this: '{opening}'\n\n"
            f"Here is an extract from the Wikipedia page:\n\n{extract}"
        )}
    ]


    # Token count for the messages
    total_tokens = 0
    for message in messages:
        message_tokens = len(enc.encode(message["content"]))
        total_tokens += message_tokens

    # Print the number of tokens used for the messages
    print(f"Total tokens for row {i + 1}: {total_tokens}")


    completion = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages
    )


    print(completion.choices[0].message)
    # Extract the generated summary
    en_summary = completion.choices[0].message.content.strip()
    en_summaries.append(en_summary)

    token_counts.append(total_tokens)

    # Add a delay after processing every 10 rows
    #if (i + 1) % 5 == 0:
    #    print("Processed 5 rows, waiting for 1 minute to avoid rate limits...")
    #    time.sleep(80)



Total tokens for row 1: 5973
ChatCompletionMessage(content='Deep in the enchanted forest, there was a curious girl named Alice. One warm afternoon, as she wandered beneath the leafy trees, she spotted something quite peculiar—a White Rabbit wearing a waistcoat and peering anxiously at a pocket watch. \n\nUtterly fascinated, Alice trailed after the rabbit, who was muttering about being late. He led her to a rabbit hole, and before Alice knew it, she was falling, falling, falling, until she gently landed in a wondrous and whimsical world.\n\nIn this topsy-turvy place, Alice found herself in a tiny room with a little door. Through its keyhole, she glimpsed the loveliest garden and longed to explore it. But there was a problem—the door was far too small for Alice to fit through!\n\nThen Alice noticed a bottle labeled "Drink me." With a sip, she magically shrank to the perfect size to slip through the door but realized she\'d forgotten the key on the table! Distressed, Alice ate a cake labe

In [107]:
en_output_csv_path = '/content/final-step3-openai.csv'

# Add the summaries as a new column to the dataframe
dff['en_concise_story'] = pd.Series(en_summaries, index=df.index[:num_rows])

# Save the updated dataframe to a new CSV file
dff.to_csv(en_output_csv_path, index=False)

# Output confirmation
print(f"CSV file enriched successfully and saved to: {en_output_csv_path}")

CSV file enriched successfully and saved to: /content/final-step3-openai.csv
